# Langchain crash course

In [1]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
import dotenv
dotenv.load_dotenv()

True

## LLMs

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Taj Mahal Bites" 


In [3]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Spice Palace"'

## Prompt Templates

In [17]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.invoke("Mexican")

{'cuisine': 'Mexican', 'text': '\n\n"Sabores de México" (Flavors of Mexico)'}

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

C:\Users\ABC\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Cantina del Sol" or "La Casa de Sabor"'

In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n"La Cantina de México"'

In [18]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Butter Chicken
2. Tandoori Chicken
3. Lamb Rogan Josh
4. Palak Paneer
5. Vegetable Biryani
6. Naan Bread
7. Samosas
8. Chicken Tikka Masala
9. Aloo Gobi
10. Mango Lassi
11. Gulab Jamun
12. Malai Kofta
13. Chicken Vindaloo
14. Dal Makhani
15. Chicken Korma
16. Vegetable Pakoras
17. Lamb Biryani
18. Chicken Biryani
19. Saag Paneer
20. Mango Chutney.


In [10]:
content = chain.invoke({"input": "Indian"})
print(content)

{'input': 'Indian', 'output': '\n\n1. Butter Chicken\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Tandoori Chicken\n5. Saag Paneer\n6. Chicken Tikka Masala\n7. Aloo Gobi\n8. Samosas\n9. Naan bread\n10. Mango Lassi\n11. Chana Masala\n12. Malai Kofta\n13. Chicken Korma\n14. Palak Pakora\n15. Gulab Jamun (dessert)'}


#### Sequential Chain

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [12]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [13]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [14]:
chain.invoke({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n"Maharaja\'s Palace" ',
 'menu_items': '\n1. Tandoori Chicken: A classic Indian dish of marinated chicken cooked in a tandoor oven.\n\n2. Mutton Biryani: Fragrant basmati rice cooked with tender pieces of mutton and aromatic spices.\n\n3. Palak Paneer: A vegetarian dish made with spinach and homemade cheese cubes in a creamy tomato-based sauce.\n\n4. Butter Chicken: Tender pieces of chicken cooked in a rich and creamy tomato-based sauce.\n\n5. Dal Makhani: Slow-cooked black lentils in a creamy and flavorful gravy.\n\n6. Rogan Josh: A flavorful Kashmiri dish of tender lamb cooked in a spicy tomato-based sauce.\n\n7. Vegetable Korma: A medley of vegetables cooked in a creamy and mildly spiced sauce.\n\n8. Naan Bread: Soft and fluffy Indian bread baked in a tandoor oven.\n\n9. Samosas: Crispy pastry pockets filled with spiced potatoes and peas.\n\n10. Mango Lassi: A refreshing yogurt-based drink blended with sweet mangoes.\n\n11. Gulab Jamun: A

## Agents

In [2]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

#### serpapi and llm-math tool

In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.invoke({"input": "What is 5 times the stock price of Tesla in year 2023 (last quarter) in United States Dollars"})



> Entering new AgentExecutor chain...
 I should use a calculator to solve this math problem
Action: Calculator
Action Input: 5 * stock price of Tesla in 2023

ValueError: LLMMathChain._evaluate("
5 * stock_price
") raised error: 'stock_price'. Please try again with a valid numerical expression

#### Wikipedia and llm-math tool

In [9]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.FEW_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.invoke({"input": "When was Elon musk born? What is his age right now in 2024?"})



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Calculator to calculate Elon Musk's age in 2024.
Action: Calculator
Action Input: 2024 - 1971
Observation: Answer: 53
Thought: I now know the final answer.
Final Answer: Elon Musk was born on June 28, 1971 and will be 53 years old in 2024.

> Finished chain.


{'input': 'When was Elon musk born? What is his age right now in 2024?',
 'output': 'Elon Musk was born on June 28, 1971 and will be 53 years old in 2024.'}

## Memory

In [20]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'text': '\n\n"El Jardín de Sabor" (The Garden of Flavor)'}


In [21]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [22]:
chain.memory

In [23]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

 

"La Casa de Sabor" (The House of Flavor)


In [25]:
name = chain.invoke({"cuisine": "Arabic"})
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Mexican\nAI:  \n\n"La Casa de Sabor" (The House of Flavor)', 'text': '\n\n"Al-Sultana\'s Palace"'}


In [26]:
print(chain.memory.buffer)

Human: Mexican
AI:  

"La Casa de Sabor" (The House of Flavor)
Human: Arabic
AI: 

"Al-Sultana's Palace"


#### ConversationChain

In [27]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [29]:
convo.invoke({"input": "Who won the first cricket world cup?"})

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was held in 1975 and was won by the West Indies. The final match was played at Lord's Cricket Ground in London, England and the West Indies beat Australia by 17 runs. The man of the match was Clive Lloyd and the top scorer was Viv Richards with 138 runs."}

In [30]:
convo.run("How much is 5+5?")

'  The sum of 5 and 5 is 10.'

In [31]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team, West Indies, was Clive Lloyd. He is considered one of the greatest captains in cricket history, leading the West Indies to two consecutive world cup victories in 1975 and 1979. He also had a successful career as a player, scoring over 7,500 runs and taking 114 wickets in international matches.'

In [32]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies. The final match was played at Lord's Cricket Ground in London, England and the West Indies beat Australia by 17 runs. The man of the match was Clive Lloyd and the top scorer was Viv Richards with 138 runs.
Human: How much is 5+5?
AI:   The sum of 5 and 5 is 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team, West Indies, was Clive Lloyd. He is considered one of the greatest captains in cricket history, leading the West Indies to two consecutive world cup victories in 1975 and 1979. He also had a successful career as a player, scoring over 7,500 runs and taking 114 wickets in international matches.


#### ConversationBufferWindowMemory

In [33]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London, England. The West Indies team won by 17 runs."

In [34]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia at Lord's Cricket Ground in London, England. The West Indies team won by 17 runs.",
 'response': '  5+5 is 10.'}

In [35]:
convo.invoke({"input": "Who was the captain of the winning team?"})

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: How much is 5+5?\nAI:   5+5 is 10.',
 'response': ' I do not have enough context to accurately answer that question. Can you provide more information?'}